### Problem 1

(100 points) Consider the following problem. 

$$
\begin{eqnarray*}
\hbox{min } f&=& x_1^2+(x_2-3)^2\\
\hbox{s.t. } g_1&=& x_2^2-2x_1\leq 0\\
             g_2&=& (x_2-1)^2+5x_1-15\leq 0\\
\end{eqnarray*}
$$

Implement an SQP algorithm with line search to solve this problem, starting from
${\bf x}_0=(1,1)^T$. Incorporate the QP subproblem. Use BFGS
approximation for the Hessian of the Lagrangian. Use the
merit function and Armijo Line Search to find the step size.

In [1]:
import numpy as np

In [205]:
# objective function
def f(x):
    return x[0][0]**2 + (x[1][0] - 3)**2

# gradient of the objective
def df(x):
    return np.array([[2*x[0][0]],
                    [2*x[1][0] - 6]])

# ineq. constraints
def constraint(x):
    return np.array([[x[1][0]**2 - 2*x[0][0]],
                    [(x[1][0] - 1)**2 + 5*x[0][0] - 15]]) 

    
# gradients of the constraints [[dg1/dx1, dg1/dx2],
#                               [dg2/dx1, dg2/dx2]]
def dg(x):
    return np.array([[-2, 2*x[1][0]],
                    [5, 2*x[1][0]-2]])
    

def line_search(x, s, mu, w_old, k):
    tilt = 0.3
    alpha = 1 # start with 1
    
    if k == 0: # at first iteration, just take the abs of mu as penalty
        w = np.abs(mu)
    else:
        w = np.zeros((2,1))
        w = np.maximum(np.abs(w), 0.5 * (w_old) + np.abs(mu))
        
    # derivative of g w.r.t. alpha d max(0, g)/da
    dg_da = np.zeros((dg(x).shape))
    idx = np.argwhere(constraint(x) <= 0)
    
    if len(idx) != 0:
        for i in range(len(idx)):
            dg_da[idx[i][0],:] = np.matmul(dg(x)[idx[i][0], :], s)
    
    #print(np.matmul(df(x), s.T))
    #print(w.T.shape, dg_da.shape)
    dF_da = np.matmul(df(x), s.T) + np.matmul(w.T, dg_da)
    
    #print(np.matmul(df(x), s.T))
    #print(np.matmul(w.T, dg_da))
    cons = constraint(x + a*s)
    cons = np.array([max(0, con) for con in cons]).reshape(cons.shape)

    F_a = lambda x, w, a, s: f(x + a*s) + np.matmul(w.T, cons)
    phi = lambda x, w, a, t, dF_da: F_a(x, w, 0, 0) + a*t*dF_da
    
    #print(F_a(x, w, 0, 0))
    #print(a*tilt*dF_da)
    while phi(x, w, a, tilt, dF_da).any() < F_a(x, w, alpha, s).any():
        alpha *= 0.8
    
    return alpha, w
            
        
        

    
def solveqp(x, W, df, g, dg):
    A0 = dg(x) # jacobian of the constraints
    b0 = g(x) # all the constraints
    mu_0 = np.zeros((b0.shape[0], 1)) # initial lagrange multipliers
    mu = []
    active_set = []
    stop = False
    while not stop:
        if len(active_set) == 0:
            #print(W.shape, df(x).shape)
            s_and_mu = np.matmul(np.linalg.inv(W.copy()), -df(x))
            s = s_and_mu[:2, :]
            mu = []
        else:
            if len(active_set) == 1:
                A = A0[active[0], :].reshape(1, -1)
                b = b0[active[0], :]
            if len(active_set) == 2:
                A = A0.copy()
                b = b0.copy()
                temp = np.vstack(np.hstack((W, A.T)), 
                                np.hstack(A, np.zeros((A.shape[0], A.shape[0]))))
                s_and_mu = np.matmul(np.linalg.inv(temp), np.vstack((-f(x).T, -b)))
                s = s_and_mu[:2, :]
                mu = s_and_mu[2:, :]
                
                if len(mu) == 1:
                    mu_0[0] = s_mu[2:3, :]
                if len(mu) == 2:
                    mu_0[0] = s_mu[2:3, :]
                    mu_0[1] = s_mu[3:, :]
            
        qp_cons = np.round((np.matmul(A0, s.reshape(-1,1))+b0))
        
        # check mu values --1 is okay, 0 is not
        mucheck = 0
        
        if len(mu) == 0: # no constraints are active
            mucheck = 1
        elif np.minimum(mu) > 0: # all constraints are valid
            mucheck = 1
            stop = True
        else: # remove most negative mu from the set
            min_idx = np.argmin(np.array(mu))
            mu.remove(np.minimum(mu))
            active_set.pop(min_idx) # also remove the corresponding entry
        
        # also add the least negative to the set
        if np.max(qp_cons) <= 0:
            if mucheck == 1:
                return s, mu_0
            else:
                idx = np.argmax(qp_constraint)
                active_set.append(idx)
                active_set = np.unique(np.array(active_set)).tolist()
        
    
            
                
def BFGS(W, x, dx, s, mu):
    dL = f(x) + np.matmul(mu.T, dg(x)) - (f(x-dx)+np.matmul(mu.T, constraint(x-dx)))
    Q = np.matmul(np.matmul(dx.T, W), dx)
    
    if np.matmul(dx.T, dL.T) >= 0.2 * np.matmul(np.matmul(dx.T, W), dx):
        theta = 1
    else:
        theta = 0.8 * Q / (Q-np.matmul(dx.T, dL.T))
        
    y = theta * dL.T + (1-theta) * np.matmul(W, dx)
    
    W_new = W + np.matmul(y, y.T) / np.matmul(y.T, s) - np.matmul(np.matmul(W, s), np.matmul(s.T, W)) / np.matmul(np.matmul(s.T, W), s) 
    
    return W_new
    
    
def sqp(f, df, g, dg, x):
    W = np.eye(x0.shape[0]) # hessian
    w = np.zeros((2,1)) # weights of the merit function for line search
    mu_old = np.zeros((x0.shape[0], 1))
    norm = np.linalg.norm(f(x0) + np.matmul(mu_old.T, dg(x0)))
    error = 1e-3 # termination creterion
    k = 0
    solution = []
    w_old = np.zeros((2, 1))
    while norm >= error:
        # solve QP sub-problem
        s, mu_new = solveqp(x, W, df, g, dg)
        
        #print(s)
        
        alpha, w_new = line_search(x, s, mu_old, w_old, k)
        w_old = w_new
        # update current solution of x
        dx = alpha*s
        x = x - dx
        # get new Hessian approx using BFGS
        W = BFGS(W, x, dx, s, mu_new)
        k += 1
        norm = np.linalg.norm(f(x) + np.matmul(mu_new.T, dg(x)))
        mu_old = mu_new
        solution.append(x)
        print(solution)
    
    return solution
        

In [ ]:
x0 = np.array([[0.], [1.]])

sol = sqp(f, df, constraint, dg, x0)

In [86]:
a = np.array([[-2],
    [0]])

In [87]:
idx = np.where(a<=0)

In [139]:
np.zeros((dg(x0).shape))

array([[0., 0.],
       [0., 0.]])

In [98]:
dg(x0)[0, :]

array([-2.,  0.])

In [147]:
np.array([max(0, con) for con in constraint(x0)]).reshape(constraint(x0).shape)

array([[0],
       [0]])

In [170]:
dg_da = np.zeros((dg(x0).shape))

In [172]:
dg_da[0, :]

array([0., 0.])

In [176]:
d1 = dg(x0)

In [177]:
d1[0, 1]

0.0

In [182]:
idx = np.where(d1<=0)

In [183]:
idx

(array([0, 0, 1], dtype=int64), array([0, 1, 1], dtype=int64))

In [185]:
np.where(d1<0)

(array([0, 1], dtype=int64), array([0, 1], dtype=int64))

In [198]:
idx = np.argwhere(d1==1)

In [195]:
d1[idx[0],:]

array([[-2.,  0.],
       [-2.,  0.]])

0